In [1]:
from IPython.display import HTML, Image, IFrame

## From numpy to tensorflow

> **miniflow**
- build a small library called miniflow to use, instead of tensorflow
- differentiable graphs: tensorflow abstraction for build networks
- backprop

## Graphs
- define neuralnet in graphs
- forward propagation


>**Neural net in terms of graphs**
- A neural network is a graph of mathematical functions such as **linear combinations** and activation functions
- The graph consists of nodes, and edges

In [4]:
Image(width=300, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2016/November/58375218_example-neural-network/example-neural-network.png')

>**Forward propagation**
- you know it

> **2 steps to build a network**
- Define the graph of nodes and edges.
- Propagate values through the graph.

## Miniflow architecture

> **Build subclass of Node**
- input node class: don't calc, just hold values
- add node class: do actual addition ops

## Forward Propagation

> **Forward pass needs two methods**
- topological_sort
- forward_pass

> **topological sort**
- you need to flatten the graph in such a way where all the input dependencies for each node are resolved before trying to run its calculation
- no need to know the details of topological sort algo
- `topological_sort()` takes in a `feed_dict`, which is how we initially set a value for an Input node. The `feed_dict` is represented by the Python dictionary data structure

In [18]:
Image(width=300, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2016/October/581037bb_topological-sort.001/topological-sort.001.jpeg')

## Learning and Loss

> **neural net improve accuracy of add over time**
- need to implement linear combination first

In [4]:
Image(width=300, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5892978b_neuron/neuron.png')

## Linear transform

In [6]:
Image(width=200, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5892a358_screen-shot-2016-10-21-at-15.43.05/screen-shot-2016-10-21-at-15.43.05.png')

> **scalar input, vector weights, vector output, vector bias**
![](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581f9571_newx/newx.png)
![](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581f9571_neww/neww.png)
$$b = [b_1, b_2, b_3 ... b_k]$$

> **vector input, matrix weights, vector output, vector bias**
![](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581f9570_newx-1n/newx-1n.png)
![](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581f956f_neww-nk/neww-nk.png)
$$b = [b_1, b_2, b_3 ... b_k]$$

> **matrix input, matrix weights, vector output, vector bias**
- but each input is processed row by row, not a whole matrix at once 
![](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/5820bdff_x-mn/x-mn.png)

## Sigmoid function

![](https://d17h27t6h515a5.cloudfront.net/topher/2016/October/58114a6a_screen-shot-2016-10-26-at-19.28.34/screen-shot-2016-10-26-at-19.28.34.png)

## Learning rate
- This is more of a guessing game than anything else but empirically values in the range 0.1 to 0.0001 work well. The range 0.001 to 0.0001 is popular, as 0.1 and 0.01 are sometimes too large.

## Final form of code

In [1]:
"""
Have fun with the number of epochs!

Be warned that if you increase them too much,
the VM will time out :)
"""

import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample
from miniflow import *

# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data by mean as 0, 1 as 1 standard deviation
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

# num of features or input nodes for training set
n_features = X_.shape[1]

# num of hidden nodes 
n_hidden = 10

# initialize W1 dimension (num-input-nodes, num-hidden-nodes) from input layer to hidden layer
W1_ = np.random.randn(n_features, n_hidden)

# set all b1 (bias for W1) to be 0, num of b1 == num of nodes on hidden_layer
b1_ = np.zeros(n_hidden)

# initialize W2, dim (num-hidden-nodes, num-output-nodes), so just 1 output node
W2_ = np.random.randn(n_hidden, 1)

# set bias from hidden layer to output layer to be 0s, num-bias == num-output-nodes
b2_ = np.zeros(1)

# Neural network
# initialize input nodes for X (training features) and y (training targets)
# X is matrix, y is vector
X, y = Input(), Input()

# initialize input nodes for W1, W2, b1 and b2
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

# initialize a Linear node for from inputs to hidden layer
l1 = Linear(X, W1, b1)

# initialize a sigmoid node from hidden layer
s1 = Sigmoid(l1)

# initialize a Linear node as output node
l2 = Linear(s1, W2, b2)

# initialize a MSE node from output node 
cost = MSE(y, l2)

# feed variable placeholders with values 
feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 1000

# Total number of examples or data points for training 
m = X_.shape[0]

# train 11 data points for each epoch
batch_size = 11

# how much batches in each epoch == how many times updates (if each batch_size update once)
steps_per_epoch = m // batch_size # try 9 // 4 == 2 or 9 // 5 == 1

# all nodes flattened into a graph
graph = topological_sort(feed_dict)

# group all weights and biases into a list 
trainables = [W1, b1, W2, b2]

# print total number of data points for training
print("Total number of examples = {}".format(m))

# Step 4: 

# for each epoch: 
for i in range(epochs):

    # set loss to be 0
    loss = 0
    
    # for each of many updates within an epoch
    for j in range(steps_per_epoch):
        
        # Step 1
        # Randomly sample a batch of examples, of size 11 or any number you set as batch_size
        # or randomize batch_size of rows of features and rows of target values 
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        # calc forward propagation
        # calc backward propagation
        forward_and_backward(graph)

        # Step 3
        # trainables are list of weights and biases
        sgd_update(trainables)

        # the last node value is loss 
        # add up loss in each iteration
        loss += graph[-1].value

    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))


Total number of examples = 506
Epoch: 1, Loss: 124.433
Epoch: 2, Loss: 33.468
Epoch: 3, Loss: 26.954
Epoch: 4, Loss: 26.468
Epoch: 5, Loss: 23.300
Epoch: 6, Loss: 19.230
Epoch: 7, Loss: 14.215
Epoch: 8, Loss: 18.586
Epoch: 9, Loss: 13.600
Epoch: 10, Loss: 14.810
Epoch: 11, Loss: 21.206
Epoch: 12, Loss: 14.295
Epoch: 13, Loss: 16.769
Epoch: 14, Loss: 12.765
Epoch: 15, Loss: 11.641
Epoch: 16, Loss: 17.200
Epoch: 17, Loss: 11.331
Epoch: 18, Loss: 10.182
Epoch: 19, Loss: 9.998
Epoch: 20, Loss: 9.992
Epoch: 21, Loss: 10.407
Epoch: 22, Loss: 11.072
Epoch: 23, Loss: 12.573
Epoch: 24, Loss: 11.438
Epoch: 25, Loss: 9.363
Epoch: 26, Loss: 12.077
Epoch: 27, Loss: 10.685
Epoch: 28, Loss: 10.814
Epoch: 29, Loss: 11.289
Epoch: 30, Loss: 9.973
Epoch: 31, Loss: 8.379
Epoch: 32, Loss: 8.187
Epoch: 33, Loss: 8.305
Epoch: 34, Loss: 9.763
Epoch: 35, Loss: 8.819
Epoch: 36, Loss: 9.805
Epoch: 37, Loss: 7.717
Epoch: 38, Loss: 9.702
Epoch: 39, Loss: 8.036
Epoch: 40, Loss: 8.846
Epoch: 41, Loss: 9.138
Epoch: 4

In [2]:
%%writefile miniflow.py

import numpy as np

class Node:
    """
    Base class for nodes in the network.

    Arguments:

        `inbound_nodes`: A list of nodes with edges into this node.
    """
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # A list of nodes (the inbound_nodes=[] above) with edges are taken as inbound_nodes into this node.
        self.inbound_nodes = inbound_nodes
        
        
        # The eventual value of this node. Set by running the forward() method.
        # when initialize this node, or right now, set value to be None
        self.value = None
        
        
        # initialize A list of nodes that this node outputs to
        # set them to be empty list 
        self.outbound_nodes = []
        
        
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        
        
        # Sets this node (we are creating at this moment) as an outbound node for all of
        # this node's inputs.
        for node in inbound_nodes:
            node.outbound_nodes.append(self)

            
    def forward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError


        
        
class Input(Node):
    """
    A generic input into the network.
    """
    def __init__(self):
        # The base class constructor has to run to set all
        # the properties here.
        #
        # The most important property on an Input is value.
        # self.value is set during `topological_sort` later.
        
        # first of all, make sure Input node is a Node class
        Node.__init__(self)

    def forward(self):
        # Do nothing because nothing is calculated.
        pass

    def backward(self):
        # An Input node has no inputs so the gradient (derivative)
        # is zero.
        # The key, `self`, is reference to this object. Yes, it is right
        self.gradients = {self: 0}
        
        
        # Weights and bias may be inputs, so you need to sum 
        # the gradient from output gradients.                      ######### yes, i see ##########
        for n in self.outbound_nodes:
            self.gradients[self] += n.gradients[self] ######## explained by computational derivation of graph ?

class Linear(Node):
    """
    Represents a node that performs a linear transform.
    """
    def __init__(self, X, W, b):
        # The base class (Node) constructor. Weights and bias
        # are treated like inbound nodes.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Performs the math behind a linear transform.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            # for l_2 node, grad_cost is MSE's gradient
            grad_cost = n.gradients[self]
            
            # Set the partial of the loss with respect to this node's inputs: ################################
            # therefore, dL2/dX * grad_cost = sum_(W) * grad_cost = sum_(W*grad_cost) 
            # therefore,                    = dot(MSE's gradient, inbound_weights.T)
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # why grad_cost first, and weight.T second? 
            # this way = we can get dim (1, num_weights_or_inputs)
            
            
            # Set the partial of the loss with respect to this node's weights.
            # therefore, dL2/dW * grad_cost = sum_(X) * grad_cost = sum_(X*grad_cost) 
            # therefore,                    = dot(inbound_X.T, MSE's gradient)
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # why X.T first, grad_cost second? 
            # this way = we can get dim (num_weights, 1)
            

            # Set the partial of the loss with respect to this node's bias.
            # dl_2/db_2 * grad_cost = sum(1) * grad_cost = sum(grad_cost)
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)

            # 1. take derivative of sigmoid with respect to l1 or first linear combination  ##################
            # 2. add up is like computional graph for adding up to different routes of gradients ################
            # multiple grade_cost = one of sigmoid's outbound_nodes' gradient, it is to inherit gradient from above #
            # https://hyp.is/yPFpUPADEea5ebOmVQr3pw/colah.github.io/posts/2015-08-Backprop/
            # 3. there is one gradient accumulation below, as there is only one input to sigmoid node ##############
            # 4. because there is no summation in the gradient, this is not dot product, just simple multiplicatin
            
            

class Sigmoid(Node):
    """
    Represents a node that performs the sigmoid activation function.
    """
    def __init__(self, node):
        # The base class constructor.
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Perform the sigmoid function and set the value.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        
        
        # Initialize the gradients to 0.  Yes, I agree
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        
        
        # Sum the partial with respect to the input over all the outputs.
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            sigmoid = self.value
            
            # 1. take derivative of sigmoid with respect to l1 or first linear combination  ##################
            # 2. add up is like computional graph for adding up to different routes of gradients ################
            # multiple grade_cost = one of sigmoid's outbound_nodes' gradient, it is to inherit gradient from above #
            # https://hyp.is/yPFpUPADEea5ebOmVQr3pw/colah.github.io/posts/2015-08-Backprop/
            # 3. there is one gradient accumulation below, as there is only one input to sigmoid node ##############
            # 4. because there is no summation in the gradient, this is not dot product, just simple multiplicatin
            self.gradients[self.inbound_nodes[0]] += sigmoid * (1 - sigmoid) * grad_cost


class MSE(Node):
    def __init__(self, y, a):
        """ ####################################
        The mean squared error cost function.
        Should be used as the last node for a network.
        
        Everything is a node: 
        There are input nodes, linear combination node, activation function node, loss function node, 
        X, y, W, b are inputs nodes too
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.

        # convert dim(3,) to dim(3,1), so 
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1) 
        a = self.inbound_nodes[1].value.reshape(-1, 1)

        # set m to be num of data points
        self.m = self.inbound_nodes[0].value.shape[0]
        # Save the computed output for backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

        
    def backward(self):
        """
        Calculates the gradient of the cost with respect to the second linear combination
    
        """
        # take derivative of MSE with respect to y
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff # for y ##########################
        
        # take derivative of MSE with respect to a or y_hat
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff # for a or y_hat ##################


# Now, we have methods or functions, instead of classes        
def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


# forward and backward pass function 
def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # graph contains all nodes (inputs, Linear, sigmoid, MSE) flattened and ordered
    # let each node do forward and backward calculation
    
    # Forward pass
    # start from the first node and do forward() on it
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    # start from the last node to the first node, and do backward() on it
    for n in graph[::-1]:
        n.backward()


def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    # Performs SGD
    #
    # Loop over the trainables [W1, b1, W2, b2]
    for t in trainables:
        # Change the trainable's value by subtracting the learning rate
        # multiplied by the partial of the cost with respect to this
        # trainable.
        
        # calc gradient of weights or biases only
        partial = t.gradients[t]
        
        # update weights or biases, it indicates partial derivative of MSE with respect to W2, W1, b1 and b2
        # in order words, gradient of W1 will inherit gradient cost all the way of the chain from W1 to Cost
        t.value -= learning_rate * partial


Overwriting miniflow.py


In [3]:
import numpy as np

In [32]:
X_ = np.array([[1,2,3,4,5], [3,2,3,4,5], [5,2,3,4,5], [7,2,3,4,5], [9,2,3,4,5]])
y_ = np.array([[2,4,5,6,7], [4,4,5,6,7], [6,4,5,6,7], [8,4,5,6,7], [10,4,5,6,7]])
batch_size = 3
X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

In [33]:
X_batch

array([[5, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [3, 2, 3, 4, 5]])

In [34]:
y_batch

array([[6, 4, 5, 6, 7],
       [2, 4, 5, 6, 7],
       [4, 4, 5, 6, 7]])

$\frac{\partial{E}}{\partial{\hat{y}}} = \frac{2}{2}\sum_x(y_x-\hat{y})$ (when take a number of data points at once) $= (y_x - \hat{y})$ when dealing with a single data point 

$\frac{\partial{C}}{\partial{l_2}} =  \frac{2}{m}\sum_x(y_x-l_2) $ (when take a number of data points at once) $=\frac{2}{1} (y_x - l_2)$ when dealing with a single data point 

$\frac{\partial{E}}{\partial{\hat{y}}} =  = \frac{2}{2}\sum_x(y_x-\hat{y})$ (when take a number of data points at once) $= (y_x - \hat{y})$ when dealing with a single data point 

$\frac{\partial{C}}{\partial{l_2}} =  \frac{2}{m}\sum_x(y_x-l_2) $ (when take a number of data points at once) $=\frac{2}{1} (y_x - l_2)$ when dealing with a single data point 
